<a href="https://colab.research.google.com/github/yskim1230/AIB_Section1-PJT_Game-Production-Plan/blob/main/n212_discussion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **N212 Generalization**

## 오늘의 목표
- 데이터를 학습/검증/테스트셋으로 분리할 수 있습니다.
- 다항 선형회귀 모델에서 최적의 차수를 찾을 수 있습니다.
- 다중 선형회귀 모델과 다항 선형회귀 모델의 결과를 비교할 수 있습니다.
- 테스트셋을 활용해 모델의 일반화 성능을 평가할 수 있습니다.

## **개념 Topic**
> 오늘은 일반화에 대해서 배웠습니다.

- 다음은 일반화 성능을 높이기 위한 몇 가지 방법입니다.
  - 아래 방법은 어떤 문제(과적합/과소적합)를 해결할 수 있는지 설명해 보세요.
  - 왜 그러한지 분산과 편향의 관점에서 설명해 보세요.
    ```markdown
    1. 학습 데이터를 추가한다.
    2. 특성의 수를 줄인다.
    3. 특성의 수를 늘린다.
    4. 차수가 높은 특성을 늘린다.
    ```
    - 2. 특성의 수를 줄인다. → 집의 크기, 방의 개수, 연식 등의 feature 수를 줄이는 것
    - 4. 차수가 높은 특성을 늘린다. → 다항회귀에서 차수를 높이면 차수가 높은 특성이 추가되는 것 

\begin{align}
y = \beta + \beta_1x + \beta_2x^2 + \beta_3x^3 + \cdots
\end{align}

- **Discussion** 표의 `정리` 탭에 답변을 정리하여 적어 주세요. 

1. 학습 데이터 추가 = 과적합 이슈를 해결하기 위함
2. 특성 수를 줄인다 = 과적합 이슈를 해결하기 위함  
3. 특성 수를 늘린다 = 과소적합 이슈를 해결하기 위함
4. 차수가 높은 특성을 늘린다 = 과소적합 이슈를 해결하기 위함

모델 복잡도↑ → 분산↑ → 과적합 가능성↑ 이고,

모델 복잡도↓ → 편향↑ → 과소적합 가능성↑



## **코딩 Topic**

### **Part.1 : 데이터 준비**

지난 노트에서도 사용했던 인도의 한 도시인 [Begaluru의 집값 데이터](https://www.kaggle.com/datasets/amitabhajoy/bengaluru-house-price-data)를 사용해서 집값을 예측하는 회귀 문제를 풀어보겠습니다.

> **Data Description**

- Area_type : Description of the area
- Availability : When it can be possessed or when it is ready
- Location : Where it is located in Bengaluru
- Size : BHK or Bedrooms
- Society : To which society it belongs
- Total_sqft : Size of the property in sq.ft
- Bath : No. of Bathrooms
- Balcony : No. of the Balcony
- Price : Value of the property in lakhs (Indian Rupee - ₹)


#### **1-1. 데이터셋 로딩 및 가공**
- 지난 노트에서 사용한 데이터를 불러옵니다.
- 지난 디스커션과 같은 방법으로 4개의 특성 선택, 결측치 및 중복값 처리, `total_sqft` 변수의 타입 변환을 수행해보세요.

In [30]:
import pandas as pd
data = pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/renewal/mldl/House_Data.csv')
display(data.head())
# 4개특성 선택
df=data[['total_sqft', 'bath', 'balcony', 'price']]
df.info()


,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,Coomee,1056,2.0,1.0,39.07
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,2600,5.0,3.0,120.00
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,NaN,1440,2.0,3.0,62.00
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,Soiewre,1521,3.0,1.0,95.00
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,NaN,1200,2.0,1.0,51.00


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13320 entries, 0 to 13319
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   total_sqft  13320 non-null  object 
 1   bath        13247 non-null  float64
 2   balcony     12711 non-null  float64
 3   price       13320 non-null  float64
dtypes: float64(3), object(1)
memory usage: 416.4+ KB


In [56]:
pd.options.display.float_format = '{:,.2f}'.format

#### **1-2. 데이터 형태 확인**
- 전처리 후, 지난 노트와 데이터의 shape이 같은지 확인해보세요.
- 특성들의 type을 확인해보세요.
  - Hint) **dtypes**

In [35]:
# Null제거
df= df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10748 entries, 0 to 10932
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   total_sqft  10748 non-null  float64
 1   bath        10748 non-null  float64
 2   balcony     10748 non-null  float64
 3   price       10748 non-null  float64
dtypes: float64(4)
memory usage: 677.9 KB


In [36]:
# 중복값제거 및 reset_index
df.duplicated().sum()
df.drop_duplicates(inplace=True)
df.reset_index(drop=True,inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10748 entries, 0 to 10747
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   total_sqft  10748 non-null  float64
 1   bath        10748 non-null  float64
 2   balcony     10748 non-null  float64
 3   price       10748 non-null  float64
dtypes: float64(4)
memory usage: 336.0 KB


In [37]:
idx = []
for i in range(len(df.total_sqft)):
  try:
    float(df.total_sqft[i])
  except:
    idx.append(i) # 수치형으로 변환되지 않은 데이터들의 index 저장. ex) 2100 - 2850 형식처럼 수치형으로 바꿀 수 없는 데이터 존재.


df.drop(idx,axis=0,inplace=True)
df.total_sqft = df.total_sqft.astype('float')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10748 entries, 0 to 10747
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   total_sqft  10748 non-null  float64
 1   bath        10748 non-null  float64
 2   balcony     10748 non-null  float64
 3   price       10748 non-null  float64
dtypes: float64(4)
memory usage: 336.0 KB


In [38]:
df.reset_index(drop=True,inplace=True)

### **Part.2 : 모델링**

#### **2-1. 데이터 분리 및 기준 모델 설정**
- 타겟은 `price`, 다른 특성들은 입력 특성으로 사용하세요.
- 데이터를 학습/테스트셋으로 분리해보세요.
  ```python
  from sklearn.model_selection import train_test_split
  ```
  - `random_state` 값으로 고정
  - `test size = 0.2`
- 분리된 학습 데이터를 학습/검증셋으로 다시 분리하세요.
  - `random_state` 값으로 고정
  - `test size = 0.2`
- 분리 후 X_train, y_train, X_val, y_val, X_test, y_test의 데이터 shape은 각각 어떻게 되나요?
- 학습 데이터의 타겟인 y_train의 평균을 사용하여 기준 모델을 설정하세요.
  - 기준 모델의 MAE는 어떻게 되나요?

In [76]:
from sklearn.model_selection import train_test_split
import numpy as np
target = 'price'
X = df.drop(target, axis=1)
y = df['price']

# 데이터를 훈련 데이터와 테스트 데이터로 나누기
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 훈련 데이터를 훈련 데이터와 검증 데이터로 나누기
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

#학습
print("X_train")
print(X_train.shape)
print("y_train")
print(y_train.shape)

#검증
print("X_val")
print(X_val.shape)
print("y_val")
print(y_val.shape)

#평가
print("X_test")
print(X_test.shape)
print("y_test")
print(y_test.shape)



X_train
(6878, 3)
y_train
(6878,)
X_val
(1720, 3)
y_val
(1720,)
X_test
(2150, 3)
y_test
(2150,)


In [53]:
import numpy as np
# 훈련 데이터 타겟의 평균값 계산
mean_price = np.mean(y_train)

# 모든 예측값이 평균값인 기준 모델 예측
y_pred_mean = np.full_like(y_val, mean_price)

# 기준 모델의 RMSE, R2 score 계산
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

rmse_baseline = np.sqrt(mean_squared_error(y_val, y_pred_mean))
r2_baseline = r2_score(y_val, y_pred_mean)
mae_simple = mean_absolute_error(y_val, y_pred_mean)

print("Baseline Model")
print("RMSE: {:.2f}".format(rmse_baseline))
print("R2 Score: {:.2f}".format(r2_baseline))
print("MAE : {:.2f}".format(r2_baseline))



Baseline Model
RMSE: 122.44
R2 Score: -0.00
MAE : -0.00


#### **2-2. 다중 선형회귀 모델**
- 모든 입력 특성을 사용해서 다중 선형회귀 모델을 만들고 학습과 검증 데이터셋에서의 MAE와 $R^2$ score를 확인해보세요.
  - MAE: `mean_absolute_error(실제값, 예측값)`
  - $R^2$: `선형회귀모델.score(입력 특성, 타겟)`
- 과적합/과소적합이 발생했나요? 발생했다면 어떻게 해결할 수 있을까요?
  - 편향과 분산의 관점에서 해결 방법을 논의해보세요.

In [59]:
from sklearn.linear_model import LinearRegression

# 다중선형회귀 모델 생성 및 학습
model = LinearRegression()
model.fit(X_train, y_train)

# 검증 데이터 예측
y_pred = model.predict(X_val)

# RMSE, R2 score 계산
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
r2 = r2_score(y_val, y_pred)
mae_multi = mean_absolute_error(y_val, y_pred_mean)

print("Multiple Linear Regression Model")
print(rmse)
print(r2)
print(mae_multi)



Multiple Linear Regression Model
92.07837867627643
0.4337062414891132
69.75144761862899


# 결론

과적합이 발생했다.

데이터의 이상치를 확인해서 분산을 줄여야 한다.



[과대적합]
- train 데이터에서의 성능은 좋지만, test 데이터에서는 성능이 나빠짐 
- train data : RMSE -low , R2- high
- test data: RMSE - high, R2-low

[과소적합]
- 훈련데이터와 테스트데이터 성능 모두 낮음
- train data: RMSE, MAE - high, R2-high
-test data: RMSE, MAE- high, R2-low

#### **2-3. 다항 선형회귀 모델**
- 모든 입력 특성을 사용해서 다항 선형회귀 모델을 만들어보세요.
  - 차수는 주어진 리스트 내에 있는 차수를 모두 사용해보세요.
  - 각 차수에 해당하는 다항 선형회귀 모델의 학습과 검증 데이터셋에서의 MAE, $R^2$ score를 확인해보세요.
- 차수가 몇일 때 과적합이 가장 심했나요?
  - 높은 차수는 낮은 차수에 비해 학습과 검증 데이터셋에서의 성능 차이가 **항상** 클까요?
- 다중 선형회귀와 다항 선형회귀 중 성능이 더 좋은 모델은 어떤 모델인가요?
  - 더 좋은 모델은 안 좋은 모델에 비해 검증 데이터에서의 MAE가 얼마나 더 낮나요?

In [61]:
polynomial_degrees = [1, 2, 3, 4, 5]

In [68]:
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

pd.options.display.float_format = '{:,.2f}'.format


def eval_models(y_true, y_pred) :
    mse = mean_squared_error(y_true, y_pred).round(3)
    rmse = np.sqrt(mse).round(3)
    mae = mean_absolute_error(y_true, y_pred).round(3)
    r2 = r2_score(y_true, y_pred)

    return mse, rmse, mae, r2

baseline = [y_train.mean()] * len(y_train)
eval_models(baseline, y_train)


from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures


          Base    Train   Validation       Test
mse  20,585.79 7,608.88 1,115,935.18 179,934.60
rmse    143.48    87.23     1,056.38     424.19
mae      73.89    42.39        78.41      53.25
r2        0.00     0.63       -73.54      -7.57


In [71]:
def PolynomialRegression(degree=5, **kwargs):
    return make_pipeline(PolynomialFeatures(degree), 
                         LinearRegression(**kwargs))

poly_ols = PolynomialRegression(degree=2)
poly_ols.fit(X_train, y_train)
y_train_pred = poly_ols.predict(X_train)

y_val_pred = poly_ols.predict(X_val)
y_test_pred = poly_ols.predict(X_test)

comparison = pd.DataFrame(index=['mse', 'rmse', 'mae', 'r2'], columns=['Base', 'Train', 'Validation', 'Test'])
comparison['Base'] = eval_models(y_train, baseline)
comparison['Train'] = eval_models(y_train, y_train_pred)
comparison['Validation'] = eval_models(y_val, y_val_pred)
comparison['Test'] = eval_models(y_test, y_test_pred)

print(comparison)

          Base    Train  Validation      Test
mse  20,585.79 9,474.97    7,408.82 13,733.55
rmse    143.48    97.34       86.07    117.19
mae      73.89    46.17       44.70     47.10
r2        0.00     0.54        0.51      0.35


In [72]:
def PolynomialRegression(degree=5, **kwargs):
    return make_pipeline(PolynomialFeatures(degree), 
                         LinearRegression(**kwargs))

poly_ols = PolynomialRegression(degree=3)
poly_ols.fit(X_train, y_train)
y_train_pred = poly_ols.predict(X_train)

y_val_pred = poly_ols.predict(X_val)
y_test_pred = poly_ols.predict(X_test)

comparison = pd.DataFrame(index=['mse', 'rmse', 'mae', 'r2'], columns=['Base', 'Train', 'Validation', 'Test'])
comparison['Base'] = eval_models(y_train, baseline)
comparison['Train'] = eval_models(y_train, y_train_pred)
comparison['Validation'] = eval_models(y_val, y_val_pred)
comparison['Test'] = eval_models(y_test, y_test_pred)

print(comparison)

          Base    Train  Validation       Test
mse  20,585.79 8,645.37    8,491.44 129,730.19
rmse    143.48    92.98       92.15     360.18
mae      73.89    43.37       43.50      51.59
r2        0.00     0.58        0.43      -5.18


In [73]:
poly_ols = PolynomialRegression(degree=4)
poly_ols.fit(X_train, y_train)
y_train_pred = poly_ols.predict(X_train)

y_val_pred = poly_ols.predict(X_val)
y_test_pred = poly_ols.predict(X_test)

comparison = pd.DataFrame(index=['mse', 'rmse', 'mae', 'r2'], columns=['Base', 'Train', 'Validation', 'Test'])
comparison['Base'] = eval_models(y_train, baseline)
comparison['Train'] = eval_models(y_train, y_train_pred)
comparison['Validation'] = eval_models(y_val, y_val_pred)
comparison['Test'] = eval_models(y_test, y_test_pred)

print(comparison)

          Base    Train  Validation       Test
mse  20,585.79 7,395.29  183,143.95 256,039.49
rmse    143.48    86.00      427.95     506.00
mae      73.89    40.50       55.01      55.55
r2        0.00     0.64      -11.23     -11.20


In [74]:
poly_ols = PolynomialRegression(degree=5)
poly_ols.fit(X_train, y_train)
y_train_pred = poly_ols.predict(X_train)

y_val_pred = poly_ols.predict(X_val)
y_test_pred = poly_ols.predict(X_test)

comparison = pd.DataFrame(index=['mse', 'rmse', 'mae', 'r2'], columns=['Base', 'Train', 'Validation', 'Test'])
comparison['Base'] = eval_models(y_train, baseline)
comparison['Train'] = eval_models(y_train, y_train_pred)
comparison['Validation'] = eval_models(y_val, y_val_pred)
comparison['Test'] = eval_models(y_test, y_test_pred)

print(comparison)

          Base    Train   Validation       Test
mse  20,585.79 7,608.88 1,115,935.18 179,934.60
rmse    143.48    87.23     1,056.38     424.19
mae      73.89    42.39        78.41      53.25
r2        0.00     0.63       -73.54      -7.57


#### **2-4. 기준 모델과 성능 비교**
- 모델링 결과 성능이 가장 좋은 모델을 최종 모델로 설정해보세요.
  - 다중 선형회귀 모델 OR 다항 선형회귀 모델(차수가 몇일 때)
- 최종 모델은 기준 모델보다 성능이 좋나요?
  - 성능을 평가하는 용도로 테스트셋의 MAE를 활용해보세요.
  - 최종 모델과 기준 모델의 성능을 비교해보세요.


In [77]:
# 2차수
poly_ols = PolynomialRegression(degree=2)
poly_ols.fit(X_train, y_train)
y_train_pred = poly_ols.predict(X_train)

y_val_pred = poly_ols.predict(X_val)
y_test_pred = poly_ols.predict(X_test)

comparison = pd.DataFrame(index=['mse', 'rmse', 'mae', 'r2'], columns=['Base', 'Train', 'Validation', 'Test'])
comparison['Base'] = eval_models(y_train, baseline)
comparison['Train'] = eval_models(y_train, y_train_pred)
comparison['Validation'] = eval_models(y_val, y_val_pred)
comparison['Test'] = eval_models(y_test, y_test_pred)

print(comparison)

# 3차수 
poly_ols = PolynomialRegression(degree=3)
poly_ols.fit(X_train, y_train)
y_train_pred = poly_ols.predict(X_train)

y_val_pred = poly_ols.predict(X_val)
y_test_pred = poly_ols.predict(X_test)

comparison = pd.DataFrame(index=['mse', 'rmse', 'mae', 'r2'], columns=['Base', 'Train', 'Validation', 'Test'])
comparison['Base'] = eval_models(y_train, baseline)
comparison['Train'] = eval_models(y_train, y_train_pred)
comparison['Validation'] = eval_models(y_val, y_val_pred)
comparison['Test'] = eval_models(y_test, y_test_pred)

print(comparison)

          Base    Train  Validation      Test
mse  18,806.88 9,452.14    7,610.80 11,719.47
rmse    137.14    97.22       87.24    108.26
mae      72.03    44.65       43.26     46.16
r2       -0.00     0.50        0.55      0.53
          Base    Train  Validation      Test
mse  18,806.88 8,832.47  155,272.25 12,203.22
rmse    137.14    93.98      394.05    110.47
mae      72.03    43.45       51.06     45.76
r2       -0.00     0.53       -8.10      0.51


#### **Optional) 과적합/과소적합을 해소하기 위한 방법을 코드로 구현해보세요.** 
  - 새로운 특성 만들기, 다항회귀 차수 조정 등등
  - 과적합/과소적합이 해소 되었나요?

### **Conclusion**
> 오늘 Topic을 수행한 결과를 바탕으로, 다음 사항에 대해 답해 주세요.

- 데이터를 학습, 검증, 테스트셋으로 분리하는 이유가 무엇일까요?

 - 답 : 일반화 성능이 개선되었는지 확인하기 위해 학습, 검증으로 분리하고 최종적으로 개선이 됐음을 확인하기 위해 테스트 셋으로 분리한다.

- 다중 선형회귀와 다항 선형회귀 모델 중 더 좋은 모델은 무엇인가요?

 - 답 : 다중 선형에서 다항 선형회귀를 비교할 때, MAE 이 작을수록, R2는 높을수록 개선이 됐다고 판단한다.
다항 선형회귀에서 MAE가 낮아지고, R2가 높았다

- 다항 선형회귀 모델의 최적의 차수는 몇인가요?

 - 답 : 최적의 차수는 2차수입니다.
2차수에서 3차수로 올라갈때 r2값이 감소했다

- **Discussion** 표의 `정리` 탭에 답변을 정리하여 적어 주세요. 